In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.src.metrics import accuracy


In [2]:
from folktables import ACSDataSource, ACSPublicCoverage
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')

# Load Data for All US States

In [ ]:
# List of all 50 US state abbreviations
states = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]
# states = ["CA", "TX"]  # Example subset of states

# Dictionary to store data, features, and labels for each state
state_data = {}
state_features = {}
state_labels = {}
state_groups = {}

# Assuming `data_source` and `ACSIncome` are already defined
for state in states:
    # Load data for each state
    state_data[state] = data_source.get_data(states=[state], download=True)
    # Convert to features and labels
    state_features[state], state_labels[state], state_groups[state]= ACSPublicCoverage.df_to_numpy(state_data[state])
    state_features[state] = scaler.fit_transform(state_features[state])
    state_groups[state] = np.where(state_groups[state] == 1, 0, 1)
    print(state, state_features[state].shape, state_labels[state].shape,state_groups[state].shape)

# Example usage: Access data, features, and labels for California (CA)



# Split Data into Train, Validation, and Test 

In [ ]:
from sklearn.model_selection import train_test_split
# Define dictionaries to store train, validation, and test data for each state
train_features = {}
train_labels = {}
train_groups = {}
val_features = {}
val_labels = {}
vali_groups = {}
test_features = {}
test_labels = {}
test_groups = {}
all_test_features = []
all_test_labels = []
all_test_groups = []
all_vaild_features = []
all_valid_labels = []
all_valid_groups = []
from sklearn.model_selection import train_test_split


# Split ratio configuration
train_ratio = 0.5
val_ratio = 0.3
test_ratio = 0.2

for state in states:
    # Get the features and labels for the state
    features = state_features[state]
    labels = state_labels[state]
    groups = state_groups[state]

    # First split into train and temp (validation + test) sets
    X_train, X_temp, y_train, y_temp, s_train, s_temp = train_test_split(
        features, labels, groups, test_size=(1 - train_ratio), random_state=42)

    # Then split the temp set into validation and test sets
    X_val, X_test, y_val, y_test, s_val, s_test = train_test_split(
        X_temp, y_temp, s_temp, test_size=(test_ratio / (val_ratio + test_ratio)), random_state=42)

    # Store the splits into respective dictionaries
    train_features[state] = X_train
    train_labels[state] = y_train
    train_groups[state] = s_train
    val_features[state] = X_val
    val_labels[state] = y_val
    vali_groups[state] = s_val
    test_features[state] = X_test
    test_labels[state] = y_test
    test_groups[state] = s_test
    all_vaild_features.append(X_val)
    all_valid_labels.append(y_val)
    all_valid_groups.append(s_val)
    all_test_features.append(X_test)
    all_test_labels.append(y_test)
    all_test_groups.append(s_test)

    # Print the shapes for verification
    print(f"{state}: Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
all_test_features = np.concatenate(all_test_features)
all_test_labels = np.concatenate(all_test_labels)
all_test_groups = np.concatenate(all_test_groups)
all_vaild_features=np.concatenate(all_vaild_features)
all_vaild_labels=np.concatenate(all_valid_labels)
all_valid_groups=np.concatenate(all_valid_groups)




# Train a FedAvg model

In [ ]:
import tensorflow as tf
import numpy as np
import time

# List of all 50 US state abbreviations

# Define a simple neural network model
def create_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Training parameters
global_epochs = 10
local_epochs = 1
batch_size = 256

# Initialize global model
input_shape = state_features[states[0]].shape[1:]  # Assuming all states have the same feature shape
global_model = create_model(input_shape)

# Federated Averaging
for epoch in range(global_epochs):
    start_time=time.time()
    print(f"Global Epoch {epoch + 1}/{global_epochs}")
    
    # Store the weights from each local model
    local_weights = []

    # Train model on each state's data
    for state in states:
        # print(f"Training on state: {state}")

        # Create a local model for each state
        local_model = create_model(input_shape)
        local_model.set_weights(global_model.get_weights())  # Initialize with global model weights

        # Train local model
        local_model.fit(
            state_features[state], state_labels[state],
            epochs=local_epochs, batch_size=batch_size
        )

        # Append the local weights
        local_weights.append(local_model.get_weights())

    # Average the weights to get new global weights
    new_global_weights = [np.mean([local_weights[j][i] for j in range(len(local_weights))], axis=0)
                          for i in range(len(local_weights[0]))]
    end_time=time.time()
    # print('time:',end_time-start_time)
    # Set the new global weights
    global_model.set_weights(new_global_weights)
    loss,accuracy=global_model.evaluate(all_vaild_features,all_vaild_labels)
    print('validation accuray:',accuracy)
# Evaluate the global model
print("\nEvaluating Global Model")
for state in states:
    loss, accuracy = global_model.evaluate(val_features[state], val_labels[state], verbose=0)
    print(f"{state} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")




# local and global accuracy, fairness (EOp) of FedAvg

In [ ]:
y_val_pred={}
local_EO=[]
local_acc=[]
def compute_equal_odd(y_true, y_pred, groups):
    white_tpr_1 = np.mean(y_pred[(y_true == 1) & (groups == 0)])
    black_tpr_1 = np.mean(y_pred[(y_true == 1) & (groups == 1)])
    white_tpr_0 = np.mean((1-y_pred)[(y_true == 0) & (groups == 0)])
    black_tpr_0 = np.mean((1-y_pred)[(y_true == 0) & (groups == 1)])
    EO=np.sum([abs(white_tpr_1-black_tpr_1),abs(white_tpr_0-black_tpr_0)])
    if EO<1.01:
        return EO
    else:
        return 0
def compute_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)
for state in states:
    y_val_pred[state]=global_model.predict(val_features[state])
    y_val_pred[state]=np.where(y_val_pred[state]<0.5,0,1)
    EO=compute_equal_odd(val_labels[state],y_val_pred[state],vali_groups[state])
    acc=compute_accuracy(val_labels[state],y_val_pred[state])
    local_acc.append(acc)
    local_EO.append(EO)
all_valid_pred=global_model.predict(all_vaild_features)
all_valid_pred=np.where(all_valid_pred<0.5,0,1)
global_EO=compute_equal_odd(all_vaild_labels,all_valid_pred,all_valid_groups)
print('global EO',global_EO)
print('local EO',local_EO)
local_EO_avg=np.mean(local_EO)
print('local EO average',local_EO_avg)
acc_diff_1=abs(max(local_acc)-min(local_acc))
print('acc diff:',acc_diff_1)


In [7]:
acc=global_model.evaluate(all_vaild_features,all_vaild_labels)
print('loss, accuracy:',acc)    

10532/10532 [==============================] - 5s 488us/step - loss: 0.4907 - accuracy: 0.7743
loss, accuracy: [0.4906599521636963, 0.7742937803268433]


# Compute the local statistics for LP

In [8]:
states_for_local_fairness = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI"
]

In [9]:
# def p_computation(label, sensitive_attribute, y, a):
#     label=np.reshape(label,(-1,))
#     N=len(label)
#     sensitive_attribute=np.reshape(sensitive_attribute,(-1,))# Total number of samples
#     # Create a boolean mask for where both conditions are met
#     mask = (label == y) & (sensitive_attribute == a)
#     # print(mask)
#     # print("Number of matching samples:", np.sum(mask))
    
#     # Compute the probability
#     p_y_ac = np.sum(mask) / N
#     return p_y_ac





import numpy as np

def p_computation(label, sensitive_attribute, y, a):
    """
    Computes a differentially private estimate of P(y, a | c)
    by adding Laplace noise.

    Parameters:
    - label: array-like, predicted or actual labels
    - sensitive_attribute: array-like, sensitive attributes (e.g., gender, race)
    - y: target label value to compute probability for
    - a: target sensitive attribute value
    - epsilon: privacy budget (smaller = more privacy, more noise)

    Returns:
    - p_y_ac_noisy: Noisy probability estimate
    """
    epsilon = 0.01  # Privacy budg
    label = np.reshape(label, (-1,))
    sensitive_attribute = np.reshape(sensitive_attribute, (-1,))
    N = len(label)

    # Compute true probability
    mask = (label == y) & (sensitive_attribute == a)
    p_y_ac = np.sum(mask) / N

    # Sensitivity is 1/N for counting queries normalized by N
    sensitivity = 1 / N

    # Sample Laplace noise
    noise = np.random.laplace(loc=0.0, scale=sensitivity / epsilon)

    # Add noise and ensure result is clipped to [0,1]
    p_y_ac_noisy = np.clip(p_y_ac + noise, 0, 1)

    return p_y_ac_noisy
#community_1
S= np.zeros((len(states),2,2))
L=np.zeros((len(states),2,2))
p = np.zeros(len(states))
l=[]
for c,state in enumerate(states):
    p[c]=len(val_features[state])/len(all_vaild_features)
for c,state in enumerate(states):
    for a in range(2):
        for y in range(2):
            S[c,a,y] =p[c]* p_computation(val_labels[state],vali_groups[state], y, a)

            L[c,a,y] =p_computation(val_labels[state],vali_groups[state], y, a)


In [ ]:
TP= np.zeros((len(states),2,2))
def compute_TP (y_pred,label, sensitive_attribute, y, a):
    y_pred=np.reshape(y_pred,(-1,))
    label=np.reshape(label,(-1,))
    N=len(y_pred)
    sensitive_attribute=np.reshape(sensitive_attribute,(-1,))# Total number of samples
    # Create a boolean mask for where both conditions are met
    count_1= (y_pred == y) & (label == y) & (sensitive_attribute == a)
    count_2 = (label== y) & (sensitive_attribute == a)
    # print(sum(count_1),sum(count_2))

    
    # Compute the probability
    TP_y_ac = np.sum(count_1) / np.sum(count_2)
    return TP_y_ac

for c, state in enumerate(states):
    y_pred=global_model.predict(val_features[state])
    y_pred=np.where(y_pred<0.5,0,1)
    for a in range(2):
        for y in range(2):
            TP[c,a,y]=compute_TP(y_pred,val_labels[state],vali_groups[state],y,a)
# print(TP)

In [11]:
def compute_alpha(label, sensitive_attribute, y, a):
    label=np.reshape(label,(-1,))
    N=len(label)
    sensitive_attribute=np.reshape(sensitive_attribute,(-1,))# Total number of samples
    # Create a boolean mask for where both conditions are met
    mask = (label == y) & (sensitive_attribute == a)
    # print(mask)
    # print("Number of matching samples:", np.sum(mask))
    
    # Compute the probability
    alpha_y_ac = np.sum(mask) / N
    return alpha_y_ac
alpha_a_y = np.zeros((2,2))
for a in range(2):
    for y in range(2):
        alpha_a_y[a,y]=compute_alpha(all_vaild_labels,all_valid_groups,y,a)
# print(alpha_a_y)


## Solve the LP

In [ ]:
from scipy.optimize import linprog
import numpy as np
def LP_EO(e_g,e_l,e_c):
# define the objective function
    C=[]
    for c in range(len(states)):
        for a in range(2):
            for y in range(2):
                C.append(-S[c,a,y])



    #define the global fairness constraints
    N=2
    K=2
    A_1=[]
    A_2=[]
    for c in range(len(states)):
        for a in range(2):
            for y in range(2):
                if  a==0 and y==0:
                    A_1.append(-S[c,a,y]/alpha_a_y[a,y])
                    # print(S[c,a,y],alpha_a_y[a,y])
                elif  a==1 and y==0:
                    A_1.append(S[c,a,y]/alpha_a_y[a,y])
    
                else:
                    A_1.append(0)
    
    for c in range(len(states)):
        for a in range(2):
            for y in range(2):
                if c==c and a==0 and y==1:
                    A_2.append(-S[c,a,y]/alpha_a_y[a,y])
                elif c==c and a==1 and y==1:
                    A_2.append(S[c,a,y]/alpha_a_y[a,y])
                else:
                    A_2.append(0)




    # define local fairness constraints
     # Identity matrix of size 50x50, where each row is a basis vector
    A_1=np.reshape(A_1,(1,len(states)*2*2))
    A_2=np.reshape(A_2,(1,len(states)*2*2))
    # print(np.shape(A_1),np.shape(A_2))
    # To access e_i, you can use basis_vectors[i], e.g., e_0 is basis_vectors[0]
    
    # Define zero vector in R^50
    zero_vec = np.zeros((2,4))
    I_vector =np.eye(2)
    basis_vectors = np.hstack([I_vector, -I_vector])
    
    # Construct the matrix
    # First row: [e_0, e_1, 0, 0]
    rows = []
  
# Construct 50 rows dynamically
    for i in range(len(states)):
        # Initialize the blocks for this row
        row_blocks = []
        
        for j in range(len(states)):
            if i == j:  # Diagonal pattern with identity matrices
                row_blocks.append(basis_vectors)
            else:  # All other positions are zero matrices
                row_blocks.append(zero_vec)
        
        # Horizontally stack the blocks for the current row
        row = np.hstack(row_blocks)
        rows.append(row)
    
    # Combine all rows into a single matrix
    A_3 = np.vstack(rows)
    # print(A_3)


    ### client fairness constraints
    rows_1=[]
    for i in range(len(states)):
        row_blocks_1 = []
        for j in range(len(states)):
            if i == j:
                for a in range(2):
                    for y in range(2):
                        row_blocks_1.append(-((len(states)-1)/len(states))*L[j,a,y])
            else:
                for a in range(2):
                    for y in range(2):
                        row_blocks_1.append((1/len(states))*L[j,a,y])
        row_1=np.hstack(row_blocks_1)
        rows_1.append(row_1)

    A_4=np.vstack(rows_1)
    print(np.shape(A_3))
    print(np.shape(A_4))     

    # Combine rows into a single matrix
    # define the property of dervied outcome predictor
    def K_ac_compute(a,c):
        K_ac=np.zeros((3,2))
        l_ac=np.zeros((3,1))
        K_ac[0:]=[-1,-1]
        K_ac[1:]=[(1-TP[c,a,1]), TP[c,a,0]]
        K_ac[2:]=[TP[c,a,1], (1-TP[c,a,0])]
        l_ac[0:]=[-1]
        l_ac[1:]=[TP[c,a,0]]
        l_ac[2:]=[TP[c,a,1]]
        return K_ac,l_ac
    num_clients = len(states)
    block_rows = 3
    block_cols = 2
    # Define the submatrices k_01, k_11, k_02, k_12 as 3x2 matrices
    M = np.zeros((3*2*len(states),2*2*len(states)))
    l = np.zeros((3*2*len(states), 1))
    
    # Construct the matrices for all clients
    for c in range(num_clients):
        for a in range(2):
            # Compute submatrices for client c and attribute a
            K_ac, l_ac = K_ac_compute(a, c)
            
            # Calculate starting positions for placing K_ac in M
            start_row = (2 * c + a) * block_rows
            start_col = (2 * c + a) * block_cols
            
            # Place K_ac and l_ac in the appropriate block of M and l
            M[start_row:start_row + block_rows, start_col:start_col + block_cols] = K_ac
            l[start_row:start_row + block_rows] = l_ac
    
    # Verify the dimensions of the resulting matrix and vector
    # print("Shape of matrix M:", M.shape)
    # print("Shape of vector l:", l.shape)
    # print(np.shape(A_1),np.shape(A_2),np.shape(A_3),np.shape(M))
    # combine the constraints
    print(np.shape(A_1),np.shape(A_2),np.shape(A_3),np.shape(M))
    A=np.vstack((A_1,A_2,A_3, A_4,-A_1, -A_2, -A_3,-A_4,M))
    print(np.shape(A))

    b_global=e_g*np.ones((1,1))
    b_local=e_l*np.ones((len(states)*2,1))
    b_client=e_c*np.ones((len(states),1))
    # print(b_local)
    b=np.vstack((b_global,b_global, b_local,b_client,b_global,b_global, b_local,b_client,l))
    print(np.shape(b))  
    res = linprog(C, A_ub=A, b_ub=b)
    x=res.x
    print(np.shape(x))
    x=np.reshape(x,(len(states),2,2))
    print(A_4)
    
    return x,res.fun
x,fun=LP_EO(0.01,0.01,0.02)
# print('objective function:',fun)  
# acc_client=[]             
# for i in range (len(states)):
#     acc=0
#     for j in range(2):
#         for k in range(2):
#             acc+=x[i,j,k]*L[i,j,k]
#     acc_client.append(acc)
# print('acc_client:',acc_client)
# print(L)

# Clients solve the LAE and generate fair outcome prediction
 

In [28]:
beta=np.zeros((50,2,3))
for a in range(2):
        for c in range(50):
            A=np.array([[1,1,1],[TP[c,a,0],1,0],[TP[c,a,1],0,1]])
            b=np.array([1,x[c,a,0],x[c,a,1]])
            beta_ac=np.linalg.solve(A,b)
            beta[c,a,:]=beta_ac

def compute_tilde_Y(c,a, Y_hat, y_values):
    """
    Compute \widetilde{Y}_{\boldsymbol{\beta}_{ac}}(x,a,c) based on given probabilities.

    Parameters:
    - beta_ac: Dictionary with keys 'beta_0' and 'beta_y' for probabilities.
    - Y_hat: The predicted value \hat{Y}(x,a,c).
    - y_values: List or array of possible y values in \mathcal{Y}.

    Returns:
    - tilde_Y: The computed value of \widetilde{Y}.
    """
    # Extract probabilities
    beta_0 = beta[c, a, 0]  # Probability for Y_hat
    beta_y = beta[c, a, 1:] 
    # Probabilities for other y in \mathcal{Y}
    
    # Normalize probabilities
    
    # Generate a random number
    rand_val = np.random.rand()
    
    # Determine the output based on random value
    if rand_val < beta_0:
        return Y_hat
    else:
        cumulative_prob = beta_0
        for y in y_values:
            cumulative_prob += beta_y[y]
            if rand_val < cumulative_prob:
                return y
y_tilde_list = [[] for _ in range(50)] 
# Example usage
for c, state in enumerate(states):
    client_features = test_features[state]
    client_sensitive = test_groups[state]
    y_pred=global_model.predict(test_features[state])
    y_pred=np.where(y_pred<0.5,0,1)
    for i in range (len(y_pred)):
        a=int(client_sensitive[i])
        y_hat=int(y_pred[i])
        y_tilde=compute_tilde_Y(c,a,y_hat,[0,1])
        y_tilde=int(y_tilde)
        y_tilde_list[c].append(y_tilde)
# print(y_tilde_list[1])
acc_list=[]
for c, state in enumerate(states):
    y_tilde=np.array(y_tilde_list[c])
    acc=compute_accuracy(test_labels[state],y_tilde)
    acc_list.append(acc)
acc_diff=abs(max(acc_list)-min(acc_list))
print(acc_diff)

 1/74 [..............................] - ETA: 0s

/tmp/ipykernel_4043572/3366997340.py:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_hat=int(y_pred[i])


12/12 [==============================] - 0s 529us/step
0.388351509404141


## Fairness and Accuracy after post-processing

In [29]:
local_post_EO=[]
for c,state in enumerate(states):
    y_post=y_tilde_list[c]
    y_post=np.reshape(y_post,(-1,))
    y_true=test_labels[state]
    y_group=test_groups[state]
    dis=compute_equal_odd(y_true,y_post,y_group)
    local_post_EO.append(dis)
print('local post EO',local_post_EO)
local_EO=np.mean(local_post_EO)
print('local EO',local_EO)
y_tilde_list = np.concatenate(y_tilde_list)  

post_EO=compute_equal_odd(all_test_labels,y_tilde_list,all_test_groups)
print('global EO', post_EO)
#compute accuracy
count=np.where(y_tilde_list==all_test_labels,1,0)
accuracy=np.sum(count)/len(count)
print('accuracy',accuracy)

local post EO [0.030086919806439294, 0.08386782408599935, 0.031545117317950844, 0.050858266244310446, 0.034310255867296635, 0.08943677007484113, 0.06855530552604688, 0.02876208361361715, 0.01902609718678061, 0.05677448106303995, 0.12142674868990655, 0.10072248029007824, 0.029357581852720138, 0.040456345476454214, 0.04633528924456057, 0.12122043861795623, 0.028060167527992508, 0.04722254540094534, 0.0, 0.030689616158052657, 0.033118571514147654, 0.021692281397528212, 0.05956833418438212, 0.019850079530373543, 0.0008123362797156641, 0.1604081423145175, 0.07674186180984849, 0.007712223736933088, 0.22863109236882656, 0.07492171077748916, 0.056845445039375275, 0.03247390962143358, 0.053951018160315334, 0.08974934025056969, 0.053314891180647694, 0.031451001282074564, 0.05828227442861089, 0.014932672857041696, 0.17286619075131954, 0.055554736722200904, 0.13737427223034238, 0.11059940003704843, 0.04335306190397614, 0.10417421223692219, 0.1683973656865223, 0.03215559824445069, 0.033570697095112

## Print the results

In [30]:
print('before post processing')
print('local equal opportunity',local_EO_avg)
print('global equal opportunity',global_EO)
print('accuracy',acc)
print('acc diff:',acc_diff_1)
print('after post processing')
print('local equal opportunity',local_EO)
print('global equal opportunity',post_EO)
print('accuracy',accuracy)
print('acc_diff', acc_diff)


before post processing
local equal opportunity 0.09219603835592173
global equal opportunity 0.05928586036608663
accuracy 0.35789473684210527
acc diff: 0.2721233585930334
after post processing
local equal opportunity 0.06755968154238313
global equal opportunity 0.01732861871667074
accuracy 0.5752418759067565
acc_diff 0.388351509404141
